### Install necessary packages

In [8]:
!pip install langchain langchain-google-genai langchain-community python-dotenv duckduckgo-search requests ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 14.3 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.4.0
    Uninstalling lxml-5.4.0:
      Successfully uninstalled lxml-5.4.0


### Import necessary libraries

In [11]:
import requests
import os
import warnings
from google.colab import userdata
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [6]:
# Retrieve API keys from Colab's Secrets manager and set them as environment variables
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
os.environ['WEATHER_API_KEY'] = userdata.get('WEATHER_API_KEY')

# Check if keys are set
if not os.getenv('GOOGLE_API_KEY'):
    raise ValueError('GOOGLE_API_KEY not found in Colab Secrets.')
if not os.getenv('WEATHER_API_KEY'):
    raise ValueError('WEATHER_API_KEY not found in Colab Secrets.')

### Tool Defenitions

In [9]:
@tool
def get_weather_data(city: str) -> str:
  """
  This function fetches the current weather data for a given city
  """
  weather_api_key = os.getenv("WEATHER_API_KEY")
  url = f'https://api.weatherstack.com/current?access_key={weather_api_key}&query={city}'
  response = requests.get(url).json()
  return str(response)

search_tool = DuckDuckGoSearchRun()

tools = [get_weather_data, search_tool]

In [12]:
model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
    max_tokens=256
)

### Agent Creation

In [14]:
prompt = hub.pull("hwchase17/react")

/usr/local/lib/python3.12/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [15]:
agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt
)

In [16]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

### Testing Agent

In [17]:
response = agent_executor.invoke({"input": "Find the capital of Madhya Pradesh, then find it's current weather condition"})
print(response)



> Entering new AgentExecutor chain...
I need to find the capital of Madhya Pradesh first, and then I need to get the weather information for that city.
Action: duckduckgo_search
Action Input: capital of Madhya PradeshThe Indian state of Madhya Pradesh came into existence on 1 November 1956. Madhya Pradesh has various geographic regions which have no official administrative governmental status; some correspond to historic countries, states or provinces. Bhopal is the capital of Madhya Pradesh and ... Bhopal : The capital of Madhya Pradesh , Bhopal is known as the “ City of Nawabs and Lakes. Bhopal is the capital of Madhya Pradesh and ... Bhopal : The capital of Madhya Pradesh , Bhopal is known as the “ City of Nawabs and Lakes. Often referred to as the Heart of India, Madhya Pradesh is a state in central India ... Bhopal is the capital of the Indian state of Madhya Pradesh . Less-fertile red-to-yellow soils are spread over much of eastern Madhya Pradesh . ... Madhya Pradesh has a numb

In [18]:
response['output']

'The capital of Madhya Pradesh is Bhopal. The current weather in Bhopal is Light Rain With Thunderstorm, with a temperature of 26 degrees Celsius.'